In [1]:
from sentinelhub import SHConfig

# INSTANCE_ID = '9bf73740-000e-42a4-bf8c-58779116c642'
CLIENT_ID = 'e196e4db-2911-4b11-884b-aa032f23e962'
CLIENT_SECRET = 'XhR4erHRU1XoTYVpFsICIBws6JkHssHl'

config = SHConfig()

if CLIENT_ID and CLIENT_SECRET:
    config.sh_client_id = CLIENT_ID
    config.sh_client_secret = CLIENT_SECRET

if config.sh_client_id == '' or config.sh_client_secret == '':
    print("Warning! To use Sentinel Hub services, please provide the credentials (client ID and client secret).")

config.save()

C:\Users\shant\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from platform import python_version

print(python_version())

3.12.3


In [3]:
import os
import datetime
import numpy as np
import matplotlib.pyplot as plt
import json

from sentinelhub import MimeType, CRS, BBox, SentinelHubRequest, SentinelHubDownloadClient, DataCollection, bbox_to_dimensions, DownloadRequest, MosaickingOrder

In [4]:
from typing import Any

def plot_image(
    image: np.ndarray, factor: float = 1.0, clip_range: tuple[float, float] | None = None, **kwargs: Any
) -> None:
    """Utility function for plotting RGB images."""
    _, ax = plt.subplots(nrows=1, ncols=1)
    if clip_range is not None:
        ax.imshow(np.clip(image * factor, *clip_range), **kwargs)
    else:
        ax.imshow(image * factor, **kwargs)
    ax.set_xticks([])
    ax.set_yticks([])

In [5]:
# bands = 12 for all bands, 2 for only B11 and B12
# mosaicking = "SIMPLE" for single image, "TILE" for multiple images metadata

def download_data(date, bbox, area_size, config, bands=12, mosaicking="SIMPLE"):
    if (bands == 12):
        bands_str = """["B01","B02","B03","B04","B05","B06","B07","B08","B8A","B09","B11","B12"]"""
        return_str = """sample.B01, sample.B02, sample.B03, sample.B04, sample.B05, sample.B06, sample.B07, sample.B08, sample.B8A, sample.B09, sample.B11, sample.B12"""
    elif (bands == 2):
        bands_str = """["B11","B12"]"""
        return_str = """sample.B11, sample.B12"""
    
    if(mosaicking == "SIMPLE"):
        start_date = date
        end_date = date
    elif(mosaicking == "TILE"):
        start_date = date - datetime.timedelta(days=60)
        end_date = date + datetime.timedelta(days=60)

    evalscript_all_bands = f"""
    //VERSION=3

    function setup() {{
        return {{
            input: [{{
                bands: {bands_str},
                units: "REFLECTANCE"
            }}],
            output: {{
                bands: {bands},
                sampleType: "FLOAT32"
            }},
            mosaicking: Mosaicking.{mosaicking},
            harmonize: true
        }};
    }}


    function updateOutputMetadata(scenes, inputMetadata, outputMetadata) {{
        outputMetadata.userData = {{
            "norm_factor": inputMetadata.normalizationFactors,
            "tiles": scenes.tiles
        }};
    }}


    function evaluatePixel(sample) {{
        return [{return_str}];
    }}
    """
    
    if(mosaicking == "SIMPLE"):
        response = [SentinelHubRequest.output_response("default", MimeType.TIFF),
                    SentinelHubRequest.output_response("userdata", MimeType.JSON)]
    elif(mosaicking == "TILE"):
        response = [SentinelHubRequest.output_response("userdata", MimeType.JSON)]

    request_all_bands = SentinelHubRequest(
    evalscript=evalscript_all_bands,
    input_data=[
        SentinelHubRequest.input_data(
            data_collection=DataCollection.SENTINEL2_L2A,
            time_interval=(start_date, end_date),
            mosaicking_order=MosaickingOrder.MOST_RECENT
        )
    ],

    responses=response,
    bbox=bbox,
    size=area_size,
    config=config,
    )

    return request_all_bands.get_data()

In [6]:
def get_available_dates(date, lat, lon, offset_deg=0.02/2):
    center_lat = lat
    center_lon = lon
    date = datetime.datetime.strptime(date, '%Y-%m-%dT%H:%M:%SZ')
    date = date.replace(hour=0, minute=0, second=0)

    bbox = BBox(bbox=[center_lon - offset_deg, center_lat - offset_deg,
                center_lon + offset_deg, center_lat + offset_deg], crs=CRS.WGS84)
    resolution = 20
    area_size = bbox_to_dimensions(bbox, resolution=resolution)

    all_bands_response = download_data(
        date, bbox, area_size, config, 2, "TILE")

    metadata = all_bands_response[0]['tiles']
    
    # extract dates
    dates = []
    for tile in metadata:
        dates.append(tile['date'][:10])
    # remove duplicates
    dates = list(dict.fromkeys(dates))
    return dates

In [7]:
folder_path = './EMIT_methane_data/jsons/'
json_files = [pos_json for pos_json in os.listdir(
    folder_path) if pos_json.endswith('.json')]
json_data = []
for json_file in json_files:
    with open(folder_path + json_file) as f:
        data = json.load(f)
        json_data.append(data)
len(json_data)

1285

In [8]:

def download_range(json_data, start, end):
    count = 0
    for idx in range(start, end+1):
        data = json_data[idx]
        date_tmp = data['features'][0]['properties']['UTC Time Observed']
        center_lat = data['features'][0]['properties']['Latitude of max concentration']
        center_lon = data['features'][0]['properties']['Longitude of max concentration']

        offset_deg = 0.02 / 2

        date = datetime.datetime.strptime(date_tmp, '%Y-%m-%dT%H:%M:%SZ')
        date = date.replace(hour=0, minute=0, second=0)

        bbox = BBox(bbox=[center_lon - offset_deg, center_lat - offset_deg, center_lon + offset_deg, center_lat + offset_deg], crs=CRS.WGS84)

        resolution = 20
        area_size = bbox_to_dimensions(bbox, resolution=resolution)

        available_dates = get_available_dates(date.strftime('%Y-%m-%dT%H:%M:%SZ'), center_lat, center_lon)

        start_date = available_dates[len(available_dates) - 1][:10]
        end_date = available_dates[0][:10]
        folder_name = f"./data/_{idx + 1}_{start_date}_{end_date}"  # folder name is start-date_end-date
        os.makedirs(folder_name, exist_ok=True)

        for date_idx in available_dates:
            all_bands_response = download_data(date_idx, bbox, area_size, config, 2, "SIMPLE")  # download B11 and B12 bands
            image_data = all_bands_response[0]['default.tif']
            json_data_tmp = all_bands_response[0]['userdata.json']['tiles'][0]
            json_data_tmp['dimensions'] = image_data.shape
            json_data_tmp['bounding_box'] = {
                "min_lat": center_lat - offset_deg,
                "max_lat": center_lat + offset_deg,
                "min_lon": center_lon - offset_deg,
                "max_lon": center_lon + offset_deg
            }

            curr_date = date_idx[:10]
            np.save(f"{folder_name}/{curr_date}.npy", image_data)
            with open(f"{folder_name}/{curr_date}.json", "w") as f:
                json.dump(json_data_tmp, f)
        count = count + 1
        print(f"Progress: {count}/{end-start+1}  Total image count: {len(available_dates)}  Saved: {folder_name}")

# Example usage
# process_json_data(json_data, start, end)

In [9]:
# zero based indexing
download_range(json_data, 727, 899)

Progress: 1/173  Total image count: 48  Saved: ./data/_728_2023-06-07_2023-10-03
Progress: 2/173  Total image count: 47  Saved: ./data/_729_2023-06-09_2023-10-04
Progress: 3/173  Total image count: 24  Saved: ./data/_730_2023-06-09_2023-10-02
Progress: 4/173  Total image count: 24  Saved: ./data/_731_2023-06-09_2023-10-02
Progress: 5/173  Total image count: 24  Saved: ./data/_732_2023-06-09_2023-10-02
Progress: 6/173  Total image count: 23  Saved: ./data/_733_2023-06-11_2023-10-04
Progress: 7/173  Total image count: 48  Saved: ./data/_734_2023-06-09_2023-10-05
Progress: 8/173  Total image count: 48  Saved: ./data/_735_2023-06-09_2023-10-05
Progress: 9/173  Total image count: 24  Saved: ./data/_736_2023-06-13_2023-10-06
Progress: 10/173  Total image count: 24  Saved: ./data/_737_2023-06-11_2023-10-04
Progress: 11/173  Total image count: 47  Saved: ./data/_738_2023-06-10_2023-10-03
Progress: 12/173  Total image count: 47  Saved: ./data/_739_2023-06-12_2023-10-07
Progress: 13/173  Total i